## This file contains the ML Part of the data

### 1. Importing Libraries

In [1]:
import pandas as pd
import numpy as np

### 2.Importing Data

In [2]:
data = pd.read_csv("df_preprocessed.csv")
data.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Day of the Week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Reason_1                   700 non-null    int64  
 1   Reason_2                   700 non-null    int64  
 2   Reason_3                   700 non-null    int64  
 3   Reason_4                   700 non-null    int64  
 4   Month Value                700 non-null    int64  
 5   Day of the Week            700 non-null    int64  
 6   Transportation Expense     700 non-null    int64  
 7   Distance to Work           700 non-null    int64  
 8   Age                        700 non-null    int64  
 9   Daily Work Load Average    700 non-null    float64
 10  Body Mass Index            700 non-null    int64  
 11  Education                  700 non-null    int64  
 12  Children                   700 non-null    int64  
 13  Pets                       700 non-null    int64  

### 3. Target values

In [4]:
targets = np.where(data["Absenteeism Time in Hours"] > data["Absenteeism Time in Hours"].median(), 1, 0)

In [5]:
data["Excessive Absenteeism"] = targets

In [6]:
targets.sum()/ targets.shape[0]

0.45571428571428574

In [7]:
data_with_targets = data.drop(["Absenteeism Time in Hours", "Distance to Work", "Daily Work Load Average","Day of the Week"],axis=1)

In [8]:
data_with_targets.head()

,Reason_1,Reason_2,Reason_3,Reason_4,Month Value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


### 4. Input Values

In [9]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        # Store copy, with_mean, and with_std as attributes of the class
        self.copy = copy  
        self.with_mean = with_mean
        self.with_std = with_std
        # Pass copy, with_mean, and with_std as keyword arguments to StandardScaler
        self.scaler = StandardScaler(copy=copy, with_mean=with_mean,with_std=with_std)  
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [11]:
unscaled_inputs.columns.values

array(['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4', 'Month Value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [12]:
#columns_to_scale = ['Month Value','Day of the Week', 'Transportation Expense', 'Distance to Work',
#                    'Age', 'Daily Work Load Average', 'Body Mass Index','Children', 'Pets']

columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [13]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [14]:
absenteeism_scaler = CustomScaler(columns_to_scale)

In [15]:
absenteeism_scaler.fit(unscaled_inputs)

C:\Users\sahil\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3785: FutureWarning: The behavior of DataFrame.var with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return var(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


CustomScaler(columns=['Month Value', 'Transportation Expense', 'Age',
                      'Body Mass Index', 'Children', 'Pets'])

In [16]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [17]:
scaled_inputs.shape

(700, 11)

### 5. Starting ML using logistic regression

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x_train,x_test,y_train,y_test = train_test_split(scaled_inputs,targets,train_size=0.8,random_state=20)

In [20]:
print(x_train.shape,y_train.shape)

(560, 11) (560,)


In [21]:
print(x_test.shape,y_test.shape)

(140, 11) (140,)


In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
reg = LogisticRegression()

In [24]:
reg.fit(x_train,y_train)

LogisticRegression()

In [25]:
reg.score(x_train,y_train)

0.7732142857142857

In [26]:
reg.intercept_

array([-1.6469898])

In [27]:
reg.coef_

array([[ 2.80000644,  0.95174778,  3.1140605 ,  0.83835931,  0.15897713,
         0.60513709, -0.16990589,  0.27998236, -0.21017416,  0.34842434,
        -0.27721907]])

In [28]:
feature_names = unscaled_inputs.columns.values

In [29]:
summary_table = pd.DataFrame(columns=["features"], data=unscaled_inputs.columns.values)

In [30]:
summary_table["Coefficient"] = np.transpose(reg.coef_)

In [31]:
summary_table

,features,Coefficient
0,Reason_1,2.800006
1,Reason_2,0.951748
2,Reason_3,3.114061
3,Reason_4,0.838359
4,Month Value,0.158977
5,Transportation Expense,0.605137
6,Age,-0.169906
7,Body Mass Index,0.279982
8,Education,-0.210174
9,Children,0.348424


In [32]:
summary_table["Odds Ratio"] = np.exp(summary_table.Coefficient)

In [33]:
summary_table.sort_values(["Odds Ratio"], ascending=False)

,features,Coefficient,Odds Ratio
2,Reason_3,3.114061,22.512270
0,Reason_1,2.800006,16.444753
1,Reason_2,0.951748,2.590233
3,Reason_4,0.838359,2.312570
5,Transportation Expense,0.605137,1.831503
9,Children,0.348424,1.416833
7,Body Mass Index,0.279982,1.323106
4,Month Value,0.158977,1.172311
6,Age,-0.169906,0.843744
8,Education,-0.210174,0.810443


In [34]:
reg.score(x_test,y_test)

0.75

In [35]:
predicted_proba = reg.predict_proba(x_test)
predicted_proba

array([[0.71342516, 0.28657484],
       [0.5873216 , 0.4126784 ],
       [0.44016153, 0.55983847],
       [0.78163061, 0.21836939],
       [0.08407928, 0.91592072],
       [0.3348226 , 0.6651774 ],
       [0.29971206, 0.70028794],
       [0.13112385, 0.86887615],
       [0.78627908, 0.21372092],
       [0.74906578, 0.25093422],
       [0.49395555, 0.50604445],
       [0.22492002, 0.77507998],
       [0.07135527, 0.92864473],
       [0.73173354, 0.26826646],
       [0.30957854, 0.69042146],
       [0.54726422, 0.45273578],
       [0.55051921, 0.44948079],
       [0.53926379, 0.46073621],
       [0.40197149, 0.59802851],
       [0.05365482, 0.94634518],
       [0.70030387, 0.29969613],
       [0.78163061, 0.21836939],
       [0.42028246, 0.57971754],
       [0.42028246, 0.57971754],
       [0.24801464, 0.75198536],
       [0.74567806, 0.25432194],
       [0.51026557, 0.48973443],
       [0.8569309 , 0.1430691 ],
       [0.20365204, 0.79634796],
       [0.78163061, 0.21836939],
       [0.

In [36]:
predicted_proba[:,1]

array([0.28657484, 0.4126784 , 0.55983847, 0.21836939, 0.91592072,
       0.6651774 , 0.70028794, 0.86887615, 0.21372092, 0.25093422,
       0.50604445, 0.77507998, 0.92864473, 0.26826646, 0.69042146,
       0.45273578, 0.44948079, 0.46073621, 0.59802851, 0.94634518,
       0.29969613, 0.21836939, 0.57971754, 0.57971754, 0.75198536,
       0.25432194, 0.48973443, 0.1430691 , 0.79634796, 0.21836939,
       0.36947677, 0.67913195, 0.68508325, 0.52870791, 0.21836939,
       0.53505228, 0.22144744, 0.73673169, 0.40500758, 0.60504297,
       0.21072119, 0.45227108, 0.23749326, 0.39847178, 0.82763577,
       0.56771922, 0.69120847, 0.28657484, 0.2192347 , 0.2032712 ,
       0.57634482, 0.32954238, 0.6651774 , 0.26937528, 0.83323682,
       0.43484145, 0.88365871, 0.23125087, 0.33433749, 0.34451397,
       0.69915101, 0.6549938 , 0.29244583, 0.79186052, 0.20752232,
       0.26838009, 0.08710411, 0.22144744, 0.73215219, 0.30536526,
       0.22144744, 0.2900789 , 0.90443841, 0.46065771, 0.60175

In [37]:
import pickle

In [38]:
with open('model1', 'wb') as file:
    pickle.dump(reg, file)

In [39]:
with open('scaler1','wb') as file:
    pickle.dump(absenteeism_scaler, file)